# Visualizations

## Setup

In [ ]:
import matplotlib.pyplot as plt
from herbie import FastHerbie, paint, wgrib2, Herbie
# from herbie.toolbox import EasyMap, ccrs, pc
import xarray as xr
from datetime import datetime
import sys
import os.path as osp
import pandas as pd
import numpy as np
sys.path.append("../src")
from utils import Dict, read_yml, str2time, print_dict_summary, read_pkl
import ingest.retrieve_hrrr_api as ih
from viz import map_var, create_gif

In [ ]:
with open("../etc/forecast_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)
config.update({
    'start_time': "2024-12-01T00:00:00Z",
    'end_time': "2024-12-02T23:00:00Z"
})
print_dict_summary(config)

In [ ]:
## Retrieve Data
# hrrr_ds = ih.retrieve_hrrr(config)
# hrrr_ds["date_time"] = hrrr_ds["date_time"].astype("datetime64[ns]")

## Save Data
# hrrr_ds.to_netcdf("../data/hrrr_ds.nc")

## Read Data
hrrr_ds = xr.open_dataset("../data/hrrr_ds.nc")

## Viz

In [ ]:
H = Herbie("2024-01-01", product="sfc")
ds = H.xarray("(?:HGT|LAND):surface")

hrrr_ds["Ew"] = hrrr_ds.Ew.where(ds.lsm > 0)

In [ ]:
mt_time = pd.to_datetime(hrrr_ds.valid_time.values).tz_localize("UTC").tz_convert("America/Denver")

In [ ]:
tstep = 0

t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

map_var(hrrr_ds, "Ew", 
        time_step = tstep,
        legend_title = "Fuel Moisture Content (%)",
        title = f"FMC Forecast at {t} UTC"
       )

In [ ]:
# Define time steps (e.g., first 10)
time_steps = range(0, 48)  # Adjust as needed
create_gif(hrrr_ds, "Ew", time_steps, gif_path='../outputs/test_forecast.gif', duration=0.5)
